In [31]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *

We note that we are using the base field `QQ` for all computations currently

In [32]:
# Let us initialize constants and vector spaces.

N = 9
n = 3
d = 3
indeterminates = [var("y_%d%d" % ((i+1),(j+1))) for i in range(n) for j in range(n)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)
glN = MatrixSpace(QQ, N)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, d)

Defining y_11, y_12, y_13, y_21, y_22, y_23, y_31, y_32, y_33


Note $\hat{\sigma_{r}}(\mathbb{P}^{n-1} \times \mathbb{P}^{n-1}) = \mathbb{V}( (r+1)\times(r+1) \text{ minors})$

In [33]:
# Let us inititalize the list of polynomials, f
# In our case, r=3.

M = MatrixSpace(R, n)
A = M(np.array(indeterminates).reshape(-1,n))
if n <= 3:
    f = [R(A.determinant())]
else:
    f = A.minors(4)
    
grad_f = [fi.gradient() for fi in f]

In [34]:
# action of basis matrix E_ab in glN onto homogeneous polynomial f[i]
def E_action(a, b, i):
    sum = -1*indeterminates[b]*grad_f[i][a]
    return R(sum)

In [35]:
# W = span{f}

f_vec = [vp_iso.polynomial_to_vector(fi) for fi in f]
W = V.subspace(f_vec)

print(W)
p = W.dimension()

Vector space of degree 165 and dimension 1 over Rational Field
Basis matrix:
1 x 165 dense matrix over Rational Field


In [36]:

W_perp = W.complement()

print(W_perp)

g = W_perp.basis()
q = W_perp.dimension()

Vector space of degree 165 and dimension 164 over Rational Field
Basis matrix:
164 x 165 dense matrix over Rational Field


We note `M_f` $:= \tilde{M}_f$

In [37]:
# Pre-processing memoization of entries of M_f

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[sparsify(vp_iso.polynomial_to_vector(E_action(i,j,l))) for i in range(N)] for j in range(N)] for l in range(p)]

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [38]:

# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(p*l) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())
print('2*(n^2)-1 =', 2*(N)-1)

dim(ker(M_f)) = 17
2*(n^2)-1 = 17
